In [1]:
require(tidyverse)
require(ggplot2)
require(data.table)
require(viridis)
require(corrplot)
require(reshape2)
require(RSpectra)
require(viridis)
require(calecopal)
require(PNWColors)
require(corrplot)
require(reshape2)
require(ggbeeswarm)
require(ggrepel)
require(ggsignif)
require(UpSetR)
require(ggpubr)
source('Marla_function.r')
source('/home//jupyter/BRI_Figures_Final_V2/helper_function/helper_function_IHA.r')

Loading required package: tidyverse

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: data.table


Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


Loading required package: viridis

Loading required package: viridi

In [2]:
###### USER INPUTS ######

images.path <- ""
dat.path <- "/home/jupyter/BRI_Figures_Final_V2/Dataset/B_cell_clustered_flow/IHA_01089_PBMC_flow_bcell_clustered_dat.csv"
path <- ""

factors <- c("cell_type", "donor", "timepoint", "group", "isotype", "cluster", "meta", "non_B")

donors <- c("BR1025","BR1031","BR1032","BR1047","BR1050","BR1057",
            "BR2013","BR2021","BR2023","BR2039","BR2044","BR2051")

times <- c("day0", "day7", "day90")
time.colors <- c("#3E5641", "#A84D38", "#DE8A68") %>% setNames(times) 

groups <- c("BR1", "BR2")
group.colors <- c("#C8E8C4", "#34236E") %>% setNames(groups) 

active.factors <- c("CD69_pos", "CXCR3_pos")
active.colors <- pnw_palette("Bay", 2) %>% as.vector() %>% setNames(active.factors)

all.cols <- c("CD183", "CD69",	"Lin", "CD19", "CD45RB", "CD24", "CD20", "HLA_DR",	"CD27",	"CD10",	
              "CD21", "CD73", "CD95", "CD11c", "CD23", "CD32", "CD38",
              "IgG", "IgA", "IgD", "IgM")

surface.markers <- c("CD183", "CD69",	"Lin", "CD19", "CD45RB",	"CD24", "CD20",	"HLA_DR",	"CD27",	"CD10",	
                     "CD21", "CD73", "CD95", "CD11c", "CD23", "CD32", "CD38",
                     "IgG", "IgA", "IgD", "IgM")

pal <- cal_palette(name="lupinus", n=6, type="continuous")
pal2 <- cal_palette(name="canary", n=6, type="continuous")
donor.colors <- c(pal, pal2) %>% setNames(donors)

subsets.orig <- c("CD11c_Effector", "Early_Memory", "Core_Memory", "CD95_Memory", "T2MBC",
             "Transitional", "Naive", 
             "Plasma")
subsets <- c("CD27_neg_Effector", "CD27_pos_Effector", "Early_Memory", "Core_Memory", "CD95_Memory", "T2MBC",
             "Transitional", "Naive", 
             "Plasma")
subset.colors <- c("#F9B5AC", "#861D31", "#C4E2E1", "#338481", "#263FA6", "#02F2BE", "#805D93", "#CF7C63", "#323949") %>% setNames(subsets)

isotypes <- c("IgD", "IgMD", "IgM", "IgG", "IgA", "ND", "surface_Ig-")
isotype.colors <- c("#664A5B", "#C05746", "#4D7184", "#8DB979", "#012A36", "#D1D2D4", "#ED7D3B") %>% setNames(isotypes)
isotype.channels <- isotypes[c(1, 3, 4, 5)]

In [22]:
# upload data
dat <- fread(dat.path) %>%
  as.data.table() %>%
  .[non_B==T, meta:="NA"] %>% 
  .[meta=="RBposCD27neg_Early_Memory", meta:="Early_Memory"] %>%
  .[meta=="RBposCD27pos_Memory", meta:="Core_Memory"] %>% 
  .[, `:=`(donor=factor(donor, levels=donors),
           cell_type=factor(cell_type), 
           timepoint=factor(timepoint, levels=times), 
           group=factor(group, levels=groups),
           isotype=factor(isotype, levels=isotypes), 
           meta=factor(meta, levels=subsets.orig))] %>%
  setCutoffsMarkers() %>% 
  .[, `:=`(donor=factor(donor, levels=donors),
           cell_type=factor(cell_type), 
           timepoint=factor(timepoint, levels=times), 
           group=factor(group, levels=groups),
           isotype=factor(isotype, levels=isotypes), 
           meta=factor(meta, levels=subsets.orig))] %>% 
  .[meta=="CD11c_Effector" & CD27_pos==F, meta:="CD27_neg_Effector"] %>%
  .[meta=="CD11c_Effector" & CD27_pos==T, meta:="CD27_pos_Effector"] %>% 
  .[, `:=`(donor=factor(donor, levels=donors),
           cell_type=factor(cell_type), 
           timepoint=factor(timepoint, levels=times), 
           group=factor(group, levels=groups),
           isotype=factor(isotype, levels=isotypes), 
           meta=factor(meta, levels=subsets))]


[1] "Setting cutoffs CXCR3+ B cells"


Warning message:
“Removed 765 rows containing missing values or values outside the scale range
(`geom_point()`).”
Warning message:
“Removed 765 rows containing missing values or values outside the scale range
(`geom_point()`).”


[1] "Setting cutoffs CD69+ B cells"


Warning message:
“Removed 5 rows containing missing values or values outside the scale range
(`geom_point()`).”
Warning message:
“Removed 5 rows containing missing values or values outside the scale range
(`geom_point()`).”


[1] "Setting cutoffs CD27+ B cells"


Warning message:
“Removed 1 row containing missing values or values outside the scale range
(`geom_point()`).”
Warning message:
“Removed 1 row containing missing values or values outside the scale range
(`geom_point()`).”
Warning message:
“Removed 1 row containing missing values or values outside the scale range
(`geom_point()`).”
Warning message in grid.Call.graphics(C_points, x$x, x$y, x$pch, x$size):
“semi-transparency is not supported on this device: reported only once per page”
Warning message:
“Removed 1 row containing missing values or values outside the scale range
(`geom_point()`).”


In [4]:
dat<-dat[is.na(meta), meta:="non_B"]

In [5]:
df_freq<-as.data.frame(table(dat$donor,dat$timepoint,dat$meta))

In [6]:
colnames(df_freq)<-c("donor","timepoint",'celltype',"Counts")

In [7]:
df_freq<-df_freq %>% group_by(donor,timepoint) %>% mutate(frequency=Counts/sum(Counts)*100) %>% mutate(frequency_clr=clr_transform(frequency))

In [8]:
write.csv(df_freq,"Flow_Frequency.csv")

# Get Frequency of IgG in CD27 neg effector B

In [58]:
dat_subet<-dat %>% filter(meta=='CD27_neg_Effector')#,isotype %in% c("IgG", "IgA"))

In [59]:
df_freq<-as.data.frame(table(dat_subet$donor,dat_subet$timepoint,dat_subet$isotype))

In [60]:
colnames(df_freq)<-c("donor","timepoint",'Isotype',"Counts")

In [61]:
df_freq<-df_freq%>% filter(!Counts==0)

In [62]:
df_freq<-df_freq %>% group_by(donor,timepoint) %>% mutate(frequency=Counts/sum(Counts)*100) %>% mutate(frequency_clr=clr_transform(frequency))

In [63]:
write.csv(df_freq,"Flow_Frequency_CD27neg_effector_B_Isotye.csv")

# Get Frequency of IgG in CD27 pos effector B

In [64]:
dat_subet<-dat %>% filter(meta=='CD27_pos_Effector')#,isotype %in% c("IgG", "IgA"))

In [65]:
df_freq<-as.data.frame(table(dat_subet$donor,dat_subet$timepoint,dat_subet$isotype))

In [66]:
colnames(df_freq)<-c("donor","timepoint",'Isotype',"Counts")

In [67]:
df_freq<-df_freq%>% filter(!Counts==0)

In [68]:
df_freq<-df_freq %>% group_by(donor,timepoint) %>% mutate(frequency=Counts/sum(Counts)*100) %>% mutate(frequency_clr=clr_transform(frequency))

In [69]:
write.csv(df_freq,"Flow_Frequency_CD27pos_effector_B_Isotye.csv")